In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy.stats import skew, boxcox
sns.set()
warnings.filterwarnings('ignore')

In [2]:
copper_data= pd.read_csv(r'Copper_Set.csv')
copper_data

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,20210701.0,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,20210401.0,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,20210101.0,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,20210401.0,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,20210101.0,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,20210401.0,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,20210301.0,577.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181668,DE633116-D1DF-4846-982E-55EFC3658A76,20200702.0,102.4824216,30200854.0,25.0,Won,W,41.0,0.96,1220.0,0000000000000000000000000000000001000777,164141591,20200701.0,591.00
181669,A48374B1-E6DB-45F2-889A-1F9C27C099EB,20200702.0,208.0864686,30200854.0,25.0,Won,W,41.0,0.95,1500.0,0000000000000000000000000000000001000227,164141591,20200701.0,589.00
181670,91643238-5C7B-4237-9A5F-63AE3D35F320,20200702.0,4.235593568,30200854.0,25.0,Won,W,41.0,0.71,1250.0,0000000000000000000000000000000001004216,164141591,20200701.0,619.00
181671,7AFFD323-01D9-4E15-B80D-7D1B03498FC8,20200702.0,-2000,30200854.0,25.0,Won,W,41.0,0.85,1250.0,0000000000000000000000000000000001001149,164141591,20200701.0,601.00


# **Preprocessing**

In [3]:
copper_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181673 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             181671 non-null  object 
 1   item_date      181672 non-null  float64
 2   quantity tons  181673 non-null  object 
 3   customer       181672 non-null  float64
 4   country        181645 non-null  float64
 5   status         181671 non-null  object 
 6   item type      181673 non-null  object 
 7   application    181649 non-null  float64
 8   thickness      181672 non-null  float64
 9   width          181673 non-null  float64
 10  material_ref   103754 non-null  object 
 11  product_ref    181673 non-null  int64  
 12  delivery date  181672 non-null  float64
 13  selling_price  181672 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 19.4+ MB


In [4]:
copper_data.isna().sum()

id                   2
item_date            1
quantity tons        0
customer             1
country             28
status               2
item type            0
application         24
thickness            1
width                0
material_ref     77919
product_ref          0
delivery date        1
selling_price        1
dtype: int64

In [5]:
# Some rubbish values are present in ‘Material_Reference’ which starts with ‘00000’ value
print(copper_data['material_ref'].isna().sum())
copper_data[(copper_data['material_ref'].isna()==False) &(copper_data['material_ref'].str.startswith('00000'))]

77919


,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,20210401.0,1047.0
5,FE372230-29B1-4A29-BB73-CB9D3078C286,20210401.0,225.790676,30202938.0,25.0,Won,W,41.0,1.50,1265.0,0000000000000000000000000000000000102325,1668701718,20210401.0,983.0
7,AA3F18A2-A7D6-43DE-8644-F0632AE2CB68,20210401.0,113.387124,30342192.0,32.0,Won,W,41.0,0.68,1220.0,0000000000000000000000000000000001000944,611993,20210101.0,931.0
8,2851F48B-DA32-4DF3-884B-17A3100DD65A,20210401.0,69.071853,30341428.0,38.0,Won,S,10.0,0.60,1275.0,0000000000000000000000000000000000106786,1668701376,20210701.0,1363.0
9,6E4D638C-A820-4A40-836F-C9D48BC65528,20210401.0,630.626917,30202938.0,25.0,Won,W,41.0,1.50,1165.0,0000000000000000000000000000000000102325,1668701718,20210401.0,983.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181668,DE633116-D1DF-4846-982E-55EFC3658A76,20200702.0,102.4824216,30200854.0,25.0,Won,W,41.0,0.96,1220.0,0000000000000000000000000000000001000777,164141591,20200701.0,591.0
181669,A48374B1-E6DB-45F2-889A-1F9C27C099EB,20200702.0,208.0864686,30200854.0,25.0,Won,W,41.0,0.95,1500.0,0000000000000000000000000000000001000227,164141591,20200701.0,589.0
181670,91643238-5C7B-4237-9A5F-63AE3D35F320,20200702.0,4.235593568,30200854.0,25.0,Won,W,41.0,0.71,1250.0,0000000000000000000000000000000001004216,164141591,20200701.0,619.0
181671,7AFFD323-01D9-4E15-B80D-7D1B03498FC8,20200702.0,-2000,30200854.0,25.0,Won,W,41.0,0.85,1250.0,0000000000000000000000000000000001001149,164141591,20200701.0,601.0


In [6]:
# Some rubbish values are present in ‘Material_Reference’ which starts with ‘00000’ value which should be converted into null.
print(copper_data['material_ref'].isna().sum())
copper_data['material_ref'] =np.where((copper_data['material_ref'].isna()==False) &(copper_data['material_ref'].str.startswith('00000')), 
                                    np.nan, 
                                    copper_data['material_ref'])
print(copper_data['material_ref'].isna().sum())

77919
100645


In [7]:
copper_data.isna().sum()

id                    2
item_date             1
quantity tons         0
customer              1
country              28
status                2
item type             0
application          24
thickness             1
width                 0
material_ref     100645
product_ref           0
delivery date         1
selling_price         1
dtype: int64

In [16]:
copper_data.nunique()

id               181671
item_date           252
quantity tons    181673
customer           1169
country              17
status                9
item type             7
application          30
thickness           594
width              1386
material_ref      11495
product_ref          33
delivery date        28
selling_price      9795
dtype: int64

In [8]:
# The 'id' column have 181671 unique values. This column not affect model building and prediction. we have to drop the 'id' column
copper_data.drop('id', axis = 1, inplace = True)
copper_data.head(3)

,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,20210701.0,854.00
1,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,NaN,1668701718,20210401.0,1047.00
2,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,20210101.0,644.33


In [9]:
print('Number of empty cells in material_ref column :',copper_data.material_ref.isna().sum())
print('Percentage of empty cells in material_ref column :',(copper_data.material_ref.isna().sum()/copper_data.shape[0])*100)

Number of empty cells in material_ref column : 100645
Percentage of empty cells in material_ref column : 55.39898609039318


In [10]:
# Percentage of empty cells in "material_ref" column is 55.40% .
# while train model "material_ref" column leads to decrease the accuracy. we can remove the column
copper_data.drop('material_ref', axis = 1, inplace = True)
copper_data.head()

,item_date,quantity tons,customer,country,status,item type,application,thickness,width,product_ref,delivery date,selling_price
0,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,1670798778,20210701.0,854.00
1,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,1668701718,20210401.0,1047.00
2,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,628377,20210101.0,644.33


In [11]:
copper_data.isna().sum()

item_date         1
quantity tons     0
customer          1
country          28
status            2
item type         0
application      24
thickness         1
width             0
product_ref       0
delivery date     1
selling_price     1
dtype: int64

In [12]:
# comparing the Number of row count and Number of empty cells count,Number of empty cell count is less than Number of row count.
# Number of empty cell count is negligible. so, we have to remove the empty cells
copper_data.dropna(axis=0, inplace = True)
inconsistance_data=copper_data[copper_data['quantity tons']=='e']
copper_data.drop(inconsistance_data.index, inplace=True)
# To reset the index values
copper_data.reset_index(drop=True, inplace=True)

In [13]:
copper_data.shape

(181637, 12)

In [17]:
copper_data.isna().sum()

item_date        2
quantity tons    0
customer         0
country          0
status           0
item type        0
application      0
thickness        0
width            0
product_ref      0
delivery date    2
selling_price    0
dtype: int64

In [28]:
copper_data.dtypes

item_date        float64
quantity tons     object
customer         float64
country          float64
status            object
item type         object
application      float64
thickness        float64
width            float64
product_ref        int64
delivery date    float64
selling_price    float64
dtype: object

In [15]:
# TYPE COSTING

copper_data['quantity tons'] = copper_data['quantity tons'].astype(float, errors='ignore')
copper_data['customer'] = copper_data['customer'].astype(int,errors='ignore')
copper_data['country'] = copper_data['country'].astype(int, errors='ignore')
copper_data['application'] = copper_data['application'].astype(int, errors='ignore')
copper_data['thickness'] = copper_data['thickness'].astype(float, errors='ignore')
copper_data['width'] = copper_data['width'].astype(float, errors='ignore')
copper_data['selling_price'] = copper_data['selling_price'].astype(float, errors='ignore')


# Converting object type data into date format
copper_data['item_date'] = copper_data['item_date'].astype(str)
copper_data['delivery date'] = copper_data['delivery date'].astype(str)
copper_data['delivery date'] = pd.to_datetime(copper_data['delivery date'],format = '%Y%m%d.0', errors='coerce').dt.strftime('%Y/%m/%d')
copper_data['item_date'] = pd.to_datetime(copper_data['item_date'],format = '%Y%m%d.0', errors='coerce').dt.strftime('%Y/%m/%d')

copper_data

,item_date,quantity tons,customer,country,status,item type,application,thickness,width,product_ref,delivery date,selling_price
0,2021/04/01,54.151139,30156308,28,Won,W,10,2.00,1500.0,1670798778,2021/07/01,854.00
1,2021/04/01,768.024839,30202938,25,Won,W,41,0.80,1210.0,1668701718,2021/04/01,1047.00
2,2021/04/01,386.127949,30153963,30,Won,WI,28,0.38,952.0,628377,2021/01/01,644.33
3,2021/04/01,202.411065,30349574,32,Won,S,59,2.30,1317.0,1668701718,2021/01/01,768.00
4,2021/04/01,785.526262,30211560,28,Won,W,10,4.00,2000.0,640665,2021/03/01,577.00
...,...,...,...,...,...,...,...,...,...,...,...,...
181632,2020/07/02,102.482422,30200854,25,Won,W,41,0.96,1220.0,164141591,2020/07/01,591.00
181633,2020/07/02,208.086469,30200854,25,Won,W,41,0.95,1500.0,164141591,2020/07/01,589.00
181634,2020/07/02,4.235594,30200854,25,Won,W,41,0.71,1250.0,164141591,2020/07/01,619.00
181635,2020/07/02,-2000.000000,30200854,25,Won,W,41,0.85,1250.0,164141591,2020/07/01,601.00


In [32]:
copper_data.nunique()

item_date           250
quantity tons    181637
customer           1167
country              17
status                9
item type             7
application          30
thickness           594
width              1386
product_ref          33
delivery date        26
selling_price      9795
dtype: int64

In [34]:
# identifying the categorical and numerical columns
numeric_columns = ['quantity tons', 'width', 'thickness', 'selling_price']
category_columns =['item_date', 'delivery date', 'customer', 'country', 'status', 'item type', 'application', 'product_ref']

In [24]:
for col in ['country', 'status', 'item type','application']:
    print(col,copper_data[col].unique())
    print('--'*20)

country [ 28  25  30  32  38  78  27  77 113  79  26  39  40  84  80 107  89]
----------------------------------------
status ['Won' 'Draft' 'To be approved' 'Lost' 'Not lost for AM' 'Wonderful'
 'Revised' 'Offered' 'Offerable']
----------------------------------------
item type ['W' 'WI' 'S' 'Others' 'PL' 'IPL' 'SLAWR']
----------------------------------------
application [10 41 28 59 15  4 38 56 42 26 27 19 20 66 29 22 40 25 67 79  3 99  2  5
 39 69 70 65 58 68]
----------------------------------------


In [23]:
for col in ['status', 'item type']:
    print(copper_data[col].value_counts())
    print('--'*20)

status
Won                116007
Lost                34432
Not lost for AM     19568
Revised              4276
To be approved       4169
Draft                3121
Offered                53
Offerable              10
Wonderful               1
Name: count, dtype: int64
----------------------------------------
item type
W         105610
S          69205
PL          5660
Others       610
WI           524
IPL           27
SLAWR          1
Name: count, dtype: int64
----------------------------------------
